# Introdução

Neste documento faremos os testes de validação do sistema.

In [ ]:
# !pip install langchain==0.0.340 langchain --force-reinstall

In [1]:
validation_results_path = './validation_results'

In [2]:
openai_api_key = "xxxxxxxxx"
cohere_key = "xxxxxxx"

# 1. Obtenção dos Metadados do Minerva

In [3]:
import os
import json
import numpy as np

In [4]:
pdf_folder_path = "./metadata_documents"

## Arquivos de Metadados

In [5]:
filename_list = []
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".json") and not filename.endswith("_x.json"):
        filename_list.append(os.path.join(pdf_folder_path, filename))

filename_list = sorted(filename_list)

In [6]:
filename_list [:5]

['./metadata_documents/documento_.json',
 './metadata_documents/documento_276189.json',
 './metadata_documents/documento_494080.json',
 './metadata_documents/documento_547235.json',
 './metadata_documents/documento_614469.json']

## Obtenção dos Metadados

In [7]:
def get_metadata(filename_list):
    metadata_dict = {}
    
    for filename in filename_list:
        with open(filename, "r") as file: 
            metadata = json.load(file)
            _id = metadata["id"]
            metadata_dict[_id] = metadata
    
    return metadata_dict

In [8]:
metadata_dict = get_metadata(filename_list)

In [9]:
key = list(metadata_dict.keys())[10]

In [10]:
metadata_dict[key]

{'title': 'Mal-estar docente : vozes de professores em uma escola pública de Niterói',
 'authors': ['Santos, Roberta Duarte dos',
  'Carneiro, Cristiana orient.',
  'Universidade Federal do Rio de Janeiro. Instituto de Psicologia.'],
 'abstract': 'A presente pesquisa de Mestrado é dedicada a apresentar algumas considerações a respeito do mal-estar docente sob o viés psicanalítico. Para tanto, utiliza o referencial teórico de Freud e de psicanalistas contemporâneos que vêm se debruçando sobre essa questão. Partindo da escuta realizada em oficinas de rodas de conversas com professores do Instituto de Educação Professor Ismael Coutinho (IEPIC), assim também como de entrevistas individuais com os docentes, foram registradas suas queixas, impotências e impasses. Nesse material, com auxílio da revisão da literatura selecionada, procuramos mapear o mal-estar nas falas dos professores, relacionando-o ao conceito freudiano de mal-estar. A análise das transcrições de tais falas tem nos levado a 

# 2. Seleção dos Testes

In [11]:
import json
from datetime import datetime

In [12]:
# validation_tests_partition = '2024-05-13' # esse eh o oficial do artigo
validation_tests_partition = '2024-06-09'

validation_tests_path = f'./validation_tests/{validation_tests_partition}'
validation_tests_path

'./validation_tests/2024-06-09'

In [13]:
with open(f'{validation_tests_path}/test_info.json', 'r') as fp:
    test_info = json.load(fp)

In [33]:
# for x in test_info[:2]:
#     print(f"{x['abstract'][:800]} & {x['query']} \\\\\n\\hline")

# 3. Vector Search

In [15]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

from lib.retriever import Retriever
from lib.reranker import Reranker

In [16]:
print("Recuperando Embeddings...")
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

Recuperando Embeddings...


In [17]:
print("Construindo Vector Database...")
vectorstore = FAISS.load_local("./faiss_vector_store/", embeddings)

Construindo Vector Database...


In [18]:
retriever = Retriever(vectorstore)
retriever1 = Retriever(vectorstore, MINIMUM_RESULTS_PER_DOCUMENT=1)
reranker = Reranker(cohere_key=cohere_key, model="rerank-multilingual-v2.0")

# Preparação

In [19]:
def get_index_of_element(list_scores, _id):
    count = 1
    for score in list_scores:
        if score['id'] == _id:
            return count
        count += 1
    return None

# Teste Retriever

**Passo a Passo do Teste:**

...

In [20]:
import time
import copy

In [22]:
retriever_results = []

for info in test_info:
    docid = info['id']
    query = info['query']
    # similar = info['similar']
    
    start = time.time()
        
    retrieved_documents = await retriever.search(
        query, k=100, fetch_k=1000, max_k_by_document=5, filter={"type":"sentences"}
    )
    
    docs_mean = []
    docs_median = []
    docs_min = []
    for idx, docs in retrieved_documents.items():
        # removes sentence == abstract
        scores = [doc[0] for doc in docs if doc[0] > 0.05]
        texts = [doc[1].page_content for doc in docs if doc[0] > 0.05]

        mean = np.mean(scores)
        median = np.median(scores)
        minimum = np.inf if len(scores) == 0 else np.min(scores)

        docs_mean.append({"id": idx, "score": mean})
        docs_median.append({"id": idx, "score": median})
        docs_min.append({"id": idx, "score": minimum})

    docs_mean = sorted(docs_mean, key=lambda elem: elem['score'])
    docs_median = sorted(docs_median, key=lambda elem: elem['score'])
    docs_min = sorted(docs_min, key=lambda elem: elem['score'])
    
    end = time.time()
    retrieval_time = end - start
    
    start = time.time()
    
    documents = reranker.prepare_documents(metadata_dict, retrieved_documents)
    reranked_documents = reranker.rerank(query, documents, 10)
    
    end = time.time()
    reranker_time = end - start

    results_by_reranker = [{"id": res['id'], "relevance_score": res['relevance_score']} for res in reranked_documents]
        
    result = info.copy()
    result["retriever_documents"] = reranked_documents[:5].copy()
    result["retriever_results"] = {
        # "rank_by_document_mean": get_index_of_element(docs_mean, docid),
        # "rank_by_document_median": get_index_of_element(docs_median, docid),
        # "rank_by_document_min": get_index_of_element(docs_min, docid),
        # "rank_by_reranker": get_index_of_element(results_by_reranker, docid),
        "results_by_mean": docs_mean[:10].copy(),
        "results_by_median": docs_median[:10].copy(),
        "results_by_min": docs_min[:10].copy(),
        "results_by_reranker": results_by_reranker[:10].copy(),
        "retrieval_time": retrieval_time,
        "reranker_time": reranker_time
    }
    
    retriever_results.append(result)
    
    print('.', end='')

....................................................................................................

In [23]:
retriever_results[1]["query"]

'O objetivo inicial da pesquisa é avaliar os efeitos tóxicos da salinização do solo causada por eventos extremos de transgressão marinha sobre minhocas (Eisenia andrei) em um cenário de aumento do nível do mar na planície de Jacarepaguá, no Rio de Janeiro. Utilizando geotecnologias, foram identificadas as classes de solo mais vulneráveis, realizando bioensaios para investigar a toxicidade da salinização e seus impactos nos ecossistemas de solo.'

In [24]:
retriever_results[1]["similar"]

KeyError: 'similar'

In [25]:
retriever_results[1]["title"]

'Ecotoxicidade terrestre do sal marinho em cenário de aumento do nível do mar na planície costeira de Jacarepaguá, Rio de Janeiro - RJ'

In [26]:
# retriever_results[1]["retriever_documents"]

In [123]:
import pickle

In [124]:
currenttimestamp = datetime.now().strftime("%Y%m%d%H%M")
print(currenttimestamp)

with open(f'{validation_results_path}/retriever_results_{currenttimestamp}.pkl', 'wb') as fp:
    pickle.dump(retriever_results, fp, protocol=pickle.HIGHEST_PROTOCOL)

202405150915


# Análise Retriever

In [28]:
import pickle
import pandas as pd

In [29]:
# with open(f'{validation_results_path}/retriever_results_202405111922.pkl', 'rb') as fp:
# with open(f'{validation_results_path}/retriever_results_202405121536.pkl', 'rb') as fp:
with open(f'{validation_results_path}/retriever_results_202405150915.pkl', 'rb') as fp:
    retriever_results = pickle.load(fp)

In [30]:
len(retriever_results)

1000

In [31]:
retriever_results[0].keys()

dict_keys(['id', 'title', 'abstract', 'query', 'similar', 'retriever_documents', 'retriever_results'])

In [32]:
retriever_results[15]["retriever_results"]["results_by_reranker"]

[{'id': '918227', 'relevance_score': 0.9999934},
 {'id': '912618', 'relevance_score': 0.8000679},
 {'id': '921202', 'relevance_score': 0.77982527},
 {'id': '913300', 'relevance_score': 0.72749174},
 {'id': '915443', 'relevance_score': 0.56427854},
 {'id': '922097', 'relevance_score': 0.5427118}]

In [37]:
retriever_results[1].keys()

dict_keys(['id', 'title', 'abstract', 'query', 'similar', 'retriever_documents', 'retriever_results'])

In [41]:
retriever_results[1]['title']

'Gestão de emergências na indústria de óleo e gás : uma proposta de aplicação da metodologia do Incident Command System'

In [43]:
retriever_results[1]['retriever_documents'][1]

{'title': 'Aperfeiçoamento da regulação brasileira de segurança do processamento submarino de óleo e gás a partir de análise crítica e comparativa com regulações congêneres',
 'authors': ['Figueredo, Ana Karolina Muniz',
  'Souza Júnior, Maurício Bezerra de orient.',
  'Vaz Junior, Carlos André coorient.',
  'Universidade Federal do Rio de Janeiro. Escola de Química'],
 'abstract': 'Com a diminuição das reservas de petróleo facilmente acessíveis no mundo, o processamento submarino vem ganhando o interesse da indústria de óleo e gás. A utilização de equipamentos submarinos traz diferentes riscos associados devido as condições operacionais e ambientais. As unidades de processamento submarino são parte do sistema submarino e podem incluir etapas de separação, elevação, escoamento e injeção dos fluidos produzido. Atualmente, existem mais de 48 unidades de processamento submarino instaladas no Brasil. Em 2015, a Agência Nacional de Petróleo, Gás Natural e Biocombustíveis (ANP) publicou um r

In [47]:
retriever_results[1]['retriever_results']

{'results_by_mean': [{'id': '943756', 'score': 1.0833454},
  {'id': '920278', 'score': 1.8786119}],
 'results_by_median': [{'id': '943756', 'score': 1.1941775},
  {'id': '920278', 'score': 1.8611869}],
 'results_by_min': [{'id': '943756', 'score': 0.80350846},
  {'id': '920278', 'score': 1.7959509}],
 'results_by_reranker': [{'id': '943756', 'relevance_score': 0.99998003},
  {'id': '920278', 'relevance_score': 0.7088176}],
 'retrieval_time': 0.219987154006958,
 'reranker_time': 0.3584113121032715}

### Average Position

In [30]:
ranks_without_reranking = []
ranks_with_reranking = []

for query_result in retriever_results:
    doc_id = query_result["id"]
    retriever_result = query_result["retriever_results"]
    
    results_without_reranking = retriever_result["results_by_min"]
    results_with_reranking = retriever_result["results_by_reranker"]
    
    rank_without_reranking = get_index_of_element(results_without_reranking, doc_id)
    rank_with_reranking = get_index_of_element(results_with_reranking, doc_id)
    
    ranks_without_reranking.append(rank_without_reranking)
    ranks_with_reranking.append(rank_with_reranking)

In [33]:
pd.Series(ranks_without_reranking).isna().sum()

6

In [34]:
pd.Series(ranks_with_reranking).isna().sum()

6

In [35]:
pd.Series(ranks_without_reranking).describe()

count    94.000000
mean      1.212766
std       0.685837
min       1.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       5.000000
dtype: float64

In [36]:
pd.Series(ranks_with_reranking).describe()

count    94.000000
mean      1.010638
std       0.103142
min       1.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       2.000000
dtype: float64

In [37]:
# nulos
[retriever_results[idx]["id"] for idx, v in enumerate(ranks_with_reranking) if v is None]

['928288', '941372', '886308', '919665', '939636', '913782']

### Mean Reciprocal Rank

Podemos transformar os nulos em um valor, para representar um valor pequeno.

In [38]:
(1 / pd.Series(ranks_without_reranking).fillna(np.inf)).mean()

0.8736666666666666

In [39]:
(1 / pd.Series(ranks_with_reranking).fillna(np.inf)).mean()

0.935

### Métricas @ 1

In [40]:
retriever_results[0]["retriever_results"].keys()

dict_keys(['results_by_mean', 'results_by_median', 'results_by_min', 'results_by_reranker', 'retrieval_time', 'reranker_time'])

In [41]:
precision1_without_reranking = []
precision1_with_reranking = []

for query_result in retriever_results:
    doc_id = query_result["id"]
    retriever_result = query_result["retriever_results"]
    
    results_without_reranking = retriever_result["results_by_min"][:1]
    results_with_reranking = retriever_result["results_by_reranker"][:1]
    
    has_returned_without_reranking = 1 if get_index_of_element(results_without_reranking, doc_id) is not None else 0
    has_returned_with_reranking = 1 if get_index_of_element(results_with_reranking, doc_id) is not None else 0

    precision1_without_reranking.append(has_returned_without_reranking / 1)
    precision1_with_reranking.append(has_returned_with_reranking / 1)

In [42]:
pd.Series(precision1_without_reranking).mean()

0.83

In [43]:
pd.Series(precision1_with_reranking).mean()

0.93

### Métricas @ 5

In [44]:
precision5_without_reranking = []
precision5_with_reranking = []

for query_result in retriever_results:
    doc_id = query_result["id"]
    relevant_list = [*query_result["similar"], doc_id]
    retriever_result = query_result["retriever_results"]
    
    results_without_reranking = retriever_result["results_by_min"][:5]
    results_with_reranking = retriever_result["results_by_reranker"][:5]
    
    has_returned_without_reranking = np.sum([1 if get_index_of_element(results_without_reranking, relevantid) is not None else 0 for relevantid in relevant_list])
    has_returned_with_reranking = np.sum([1 if get_index_of_element(results_with_reranking, relevantid) is not None else 0 for relevantid in relevant_list])

    precision5_without_reranking.append(has_returned_without_reranking / len(relevant_list))
    precision5_with_reranking.append(has_returned_with_reranking / len(relevant_list))

KeyError: 'similar'

In [45]:
pd.Series(precision5_without_reranking).mean()

nan

In [36]:
pd.Series(precision5_with_reranking).mean()

0.48358333333333337

### Métricas R

In [46]:
precisionR_without_reranking = []
precisionR_with_reranking = []

for query_result in retriever_results:
    doc_id = query_result["id"]
    relevant_list = [*query_result["similar"], doc_id]
    retriever_result = query_result["retriever_results"]
    
    results_without_reranking = retriever_result["results_by_min"][:len(relevant_list)]
    results_with_reranking = retriever_result["results_by_reranker"][:len(relevant_list)]
    
    has_returned_without_reranking = np.sum([1 if get_index_of_element(results_without_reranking, relevantid) is not None else 0 for relevantid in relevant_list])
    has_returned_with_reranking = np.sum([1 if get_index_of_element(results_with_reranking, relevantid) is not None else 0 for relevantid in relevant_list])

    precisionR_without_reranking.append(has_returned_without_reranking / len(relevant_list))
    precisionR_with_reranking.append(has_returned_with_reranking / len(relevant_list))

NameError: name 'relevant_list' is not defined

In [47]:
pd.Series(precisionR_without_reranking).mean()

nan

In [48]:
pd.Series(precisionR_with_reranking).mean()

nan

### Context Relevancy (Top 5)

O modelo de reranker já faz uma análise de relevância para fazer o reranking, então podemos utilizar as métricas...

Isso também pode ser feito com o output final de relevancia....

In [50]:
context_relevance_with_reranking = []

for query_result in retriever_results:
    doc_id = query_result["id"]
    # relevant_list = [*query_result["similar"], doc_id]
    retriever_result = query_result["retriever_results"]

    results_with_reranking = retriever_result["results_by_reranker"][:5]

    context_relevance = [res["relevance_score"] for res in results_with_reranking]

    context_relevance_with_reranking.extend(context_relevance)

In [51]:
pd.Series(context_relevance_with_reranking).fillna(0).mean()

0.9078932495161289

In [53]:
context_relevance_without_reranking = []

for query_result in retriever_results:
    doc_id = query_result["id"]
    # relevant_list = [*query_result["similar"], doc_id]
    retriever_result = query_result["retriever_results"]
    
    score_with_reranking = {r['id']: r['relevance_score'] for r in retriever_result["results_by_reranker"]}
    results_without_reranking = retriever_result["results_by_min"][:5]
    
    # print(score_with_reranking)
    # print(results_without_reranking)

    context_relevance = [score_with_reranking.get(res["id"]) or 0.4 for res in results_without_reranking]

    context_relevance_without_reranking.extend(context_relevance)

In [54]:
pd.Series(context_relevance_without_reranking).fillna(0).mean()

0.6453237157929518

### Tempo de Execução

In [187]:
pd.Series([r['retriever_results']['retrieval_time'] for r in retriever_results]).describe()

count    1000.000000
mean        0.251965
std         0.057481
min         0.179228
25%         0.206047
50%         0.239344
75%         0.287115
max         0.714301
dtype: float64

In [188]:
pd.Series([r['retriever_results']['reranker_time'] for r in retriever_results]).describe()

count    1000.000000
mean        0.643066
std         0.846831
min         0.271600
25%         0.461383
50%         0.642182
75%         0.727551
max        26.506854
dtype: float64

# Teste Generation

In [149]:
from lib.generator import Generator
import asyncio
import time

In [153]:
# generation_results = []
# error = []

for info in retriever_results[:1000]:
    docid = info['id']
    query = info['query']
    reranked_documents = info["retriever_documents"]
    
    start = time.time()
    
    try:
        llm_results = []
        for doc in reranked_documents:
            generator = Generator(
                openai_api_key,
                query,
                document_info=doc,
                model_name="gpt-3.5-turbo-0125",
                temperature=0.2,
                max_tokens=4096
            )
            await generator.run_llm()

            llm_result = {
                "id": doc["id"],
                "doc": doc,
                "relevance": generator.article_relevance,
                "summary": generator.article_summary,
                "objectives": generator.article_objectives,
                "conclusions": generator.article_conclusions
            }

            llm_results.append(llm_result)

    except Exception as e:
        print("ERROR:", e)
        error.append(docid)
        continue
    
    end = time.time()
    generation_time = end - start
        
    result = info.copy()
    result["generation_results"] = {
        "results": llm_results,
        "generation_time": generation_time
    }    
    
    generation_results.append(result)
    
    print('.', end='')

............................................................................................................................................................................................................................................................................................................................................................................................

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


......................................................................................................................................................................................................................................................................................................................................................................................................................................

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


.........................

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: HTTP code 520 from API (<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
 

.............................................................................................................................................................................

In [157]:
# generation_results[0]

In [158]:
len(generation_results)

1000

In [159]:
import pickle

In [161]:
currenttimestamp = datetime.now().strftime("%Y%m%d%H%M")
print(currenttimestamp)

with open(f'{validation_results_path}/generation_results_{currenttimestamp}.pkl', 'wb') as fp:
    pickle.dump(generation_results, fp, protocol=pickle.HIGHEST_PROTOCOL)

202405170017


# Análise Generation

In [48]:
import pickle
import pandas as pd

In [49]:
# with open(f'{validation_results_path}/generation_results_202405121153.pkl', 'rb') as fp:
with open(f'{validation_results_path}/generation_results_202405170017.pkl', 'rb') as fp:     # teste oficial 1000 abstracts
    generation_results = pickle.load(fp)

In [50]:
len(generation_results)

1000

In [51]:
generation_results[0].keys()

dict_keys(['id', 'title', 'abstract', 'query', 'similar', 'retriever_documents', 'retriever_results', 'generation_results'])

In [59]:
generation_results[0]['generation_results'].keys()

dict_keys(['results', 'generation_time'])

In [63]:
generation_results[1]['generation_results']['results'][1]

{'id': '920278',
 'doc': {'title': 'Aperfeiçoamento da regulação brasileira de segurança do processamento submarino de óleo e gás a partir de análise crítica e comparativa com regulações congêneres',
  'authors': ['Figueredo, Ana Karolina Muniz',
   'Souza Júnior, Maurício Bezerra de orient.',
   'Vaz Junior, Carlos André coorient.',
   'Universidade Federal do Rio de Janeiro. Escola de Química'],
  'abstract': 'Com a diminuição das reservas de petróleo facilmente acessíveis no mundo, o processamento submarino vem ganhando o interesse da indústria de óleo e gás. A utilização de equipamentos submarinos traz diferentes riscos associados devido as condições operacionais e ambientais. As unidades de processamento submarino são parte do sistema submarino e podem incluir etapas de separação, elevação, escoamento e injeção dos fluidos produzido. Atualmente, existem mais de 48 unidades de processamento submarino instaladas no Brasil. Em 2015, a Agência Nacional de Petróleo, Gás Natural e Bioco

In [53]:
pd.Series([r['generation_results']["generation_time"] for r in generation_results]).describe()

count    1.000000e+03
mean     5.030625e+01
std      9.506539e+02
min      2.384186e-07
25%      7.818095e+00
50%      1.560901e+01
75%      3.121361e+01
max      3.007287e+04
dtype: float64

In [54]:
np.median([r['generation_results']["generation_time"] for r in generation_results])

15.609010934829712

In [55]:
index = 15

In [47]:
generation_results[index]["title"]

'produção de um campo de exploração petrolífera por diferentes abordagens analíticas'

In [48]:
generation_results[index]["query"]

'O objetivo da pesquisa é caracterizar e comparar as regiões hipervariáveis do gene rRNA 16S da comunidade microbiana presente na água de produção de um tanque de armazenamento da plataforma P-37 da PETROBRAS, a fim de conhecer a diversidade microbiológica.'

In [49]:
len(generation_results[index]["generation_results"])

2

In [66]:
for r in generation_results:
    for rr in r["generation_results"]["results"]:
        if rr["relevance"] == "":
            print(rr)

In [50]:
# for result in generation_results[index]["generation_results"]:
#     print(f">>> {result['doc']['title']} [Relevância {result['doc']['relevance_score']}]", "\n")

#     print("> Abstract:", result["doc"]["abstract"], "\n")
#     print("> Relevance:", result["relevance"], "\n")
#     print("> Summary:", result["summary"], "\n")
#     print("> Objectives:", result["objectives"], "\n")
#     print("> Conclusions:", result["conclusions"], "\n")
          
#     print("---\n")

### Answer Relevance

Visto que a única parte da resposta que diretamente diz respeito a consulta é a relevância, será a unica analisada.

In [51]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings, HuggingFaceBgeEmbeddings

In [52]:
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

In [53]:
from langchain.evaluation import load_evaluator, EmbeddingDistance

In [54]:
evaluator = load_evaluator("pairwise_embedding_distance", embeddings=embedding_model)

In [183]:
similarity_scores = []
for res in generation_results:
    scores = []
    for gen_res in res["generation_results"]["results"]:
        simrelevance = evaluator.evaluate_string_pairs(
            prediction=res["query"],
            prediction_b=gen_res["relevance"],
            distance_metric=EmbeddingDistance.COSINE
        )['score']
        
        simsummary = evaluator.evaluate_string_pairs(
            prediction=res["query"],
            prediction_b=gen_res["summary"],
            distance_metric=EmbeddingDistance.COSINE
        )['score']
        
        simobjectives = evaluator.evaluate_string_pairs(
            prediction=res["query"],
            prediction_b=gen_res["objectives"],
            distance_metric=EmbeddingDistance.COSINE
        )['score']
        
        simconclusions = evaluator.evaluate_string_pairs(
            prediction=res["query"],
            prediction_b=gen_res["conclusions"],
            distance_metric=EmbeddingDistance.COSINE
        )['score']

        scores.append({
            "relevance": simrelevance,
            "summary": simsummary,
            "objectives": simobjectives,
            "conclusions": simconclusions
        })
    
    similarity_scores.append(scores)
    print(".", end="")

In [184]:
import functools
import operator

simple_similarity_scores = functools.reduce(operator.iconcat, similarity_scores, [])

In [138]:
np.mean([r['relevance'] for r in simple_similarity_scores])

0.15443502177333301

In [139]:
np.mean([r['summary'] for r in simple_similarity_scores])

0.1924503854766851

In [140]:
np.mean([r['objectives'] for r in simple_similarity_scores])

0.27054856187288345

In [141]:
np.mean([r['conclusions'] for r in simple_similarity_scores])

0.2707868140405986

### Faithfulness

In [68]:
generation_results[0]["generation_results"]["results"][0]["summary"]

"O artigo apresenta uma leitura da narrativa de Christy Brown em 'Meu Pé Esquerdo', abordando temas como corpo, amor, felicidade e tristeza. Analisa as diferenças entre o livro e o filme, além de explorar a relação entre leitura, interpretação, educação poética e superação humana. Destaca a importância da obra de Brown para a reflexão sobre questões relacionadas à deficiência e superação."

In [69]:
generation_results[0]["generation_results"]["results"][0]["objectives"]

"O artigo tem como objetivo analisar as questões abordadas na narrativa de Christy Brown em 'Meu Pé Esquerdo', destacando temas como corpo, amor, felicidade e tristeza, e explorar as diferenças entre o livro e o filme, além de discutir a relação entre leitura, interpretação, educação poética e superação humana."

In [70]:
generation_results[0]["generation_results"]["results"][0]["conclusions"]

'O artigo conclui que a obra de Christy Brown proporciona uma reflexão profunda sobre a vida, a deficiência, a superação e a importância da arte e da educação poética. Destaca o poder de superação inerente ao ser humano e a relevância da leitura e interpretação crítica de obras literárias e cinematográficas.'

In [71]:
def generate_faithful_check(result, result_2=None):
    # usamos o result para os contextos e result2 para as geracoes
    
    result_2 = result_2 or result
    
    paragrafos = "\n\n".join(["[CONTEXTO] \"" + s.page_content + "\"" for s in result["doc"]["sentences"]])

    query = f"""
    Um modelo de linguagem gerou alguns textos a respeito de um artigo científico a partir dos seguintes contextos:

    [CONTEXTO] "{result["doc"]["abstract"]}"

    {paragrafos}

    A partir destes contextos, é possível afirmar os textos a seguir? Todos os fatos apresentados nos seguintes textos podem ser encontrados nos contextos?

    RESUMO: {result_2["summary"]}
    OBJETIVOS: {result_2["objectives"]}
    CONCLUSÕES: {result_2["conclusions"]}

    Responda com apenas SIM ou NÃO:
    """
    
    return query

In [72]:
# print(generate_faithful_check(generation_results[0]["generation_results"]["results"][0]))

In [73]:
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI

In [74]:
openai_api_key = "xxxxxx"

In [75]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.2,
    openai_api_key=openai_api_key,       
    max_tokens=4096
)

In [76]:
import random
random.seed(42)

import time

In [98]:
faithful_results = []
error = []
bad_checks = []

for gen_res in generation_results[200:300]:
    tmp_results = []
    tmp_bad = []

    # only the top results
    for res in gen_res["generation_results"]["results"][:1]:
        ## obtem uma resposta errada
        res_fake = {"id": res["id"]}
        while res["id"] == res_fake["id"]:
            try:
                gen_res_fake = random.choice(generation_results)
                res_fake = random.choice(gen_res_fake["generation_results"]["results"])
            except:
                continue
        
        query = generate_faithful_check(res, res_fake)
        # print(query, "\n\n---\n\n")
        
        messages = [SystemMessage(content=query)]
        response = llm(messages).content
                
        if response == "SIM":
            tmp_results.append(1)
        elif response == "NÃO":
            tmp_results.append(0)
            tmp_bad.append(query)
        else:
            tmp_results.append(None)
            error.append(response)
    
    time.sleep(5)
    print(".", end="")
            
    faithful_results.append(tmp_results)
    bad_checks.append(tmp_bad)

N
..N
.N
.N
.N
..N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
..N
.N
..N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
...N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
..N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
.N
..N
.N
.N
.N
.N
.N
..N
..N
.N
.N
.N
.N
.N
.N
..N
.N
.N
.N
..

In [121]:
len(faithful_results)

100

In [122]:
import functools
import operator

simple_faithful_results = functools.reduce(operator.iconcat, faithful_results, [])

In [123]:
pd.Series(simple_faithful_results).value_counts()

0    88
1     7
Name: count, dtype: int64

# Answer Semantic Similarity

In [126]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings, HuggingFaceBgeEmbeddings

In [127]:
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

In [128]:
from langchain.evaluation import load_evaluator, EmbeddingDistance

In [129]:
evaluator = load_evaluator("pairwise_embedding_distance", embeddings=embedding_model)

In [143]:
answer_similarity_scores = []
for res in generation_results:
    scores = []
    for gen_res in res["generation_results"]["results"]:
        simrelevance = evaluator.evaluate_string_pairs(
            prediction=res["abstract"],
            prediction_b=gen_res["relevance"],
            distance_metric=EmbeddingDistance.COSINE
        )['score']
        
        simsummary = evaluator.evaluate_string_pairs(
            prediction=res["abstract"],
            prediction_b=gen_res["summary"],
            distance_metric=EmbeddingDistance.COSINE
        )['score']
        
        simobjectives = evaluator.evaluate_string_pairs(
            prediction=res["abstract"],
            prediction_b=gen_res["objectives"],
            distance_metric=EmbeddingDistance.COSINE
        )['score']
        
        simconclusions = evaluator.evaluate_string_pairs(
            prediction=res["abstract"],
            prediction_b=gen_res["conclusions"],
            distance_metric=EmbeddingDistance.COSINE
        )['score']

        scores.append({
            "relevance": simrelevance,
            "summary": simsummary,
            "objectives": simobjectives,
            "conclusions": simconclusions
        })
    
    answer_similarity_scores.append(scores)
    print(".", end="")

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [144]:
len(answer_similarity_scores)

1000

In [145]:
import functools
import operator

simple_answer_similarity_scores = functools.reduce(operator.iconcat, answer_similarity_scores, [])

In [146]:
np.mean([r['relevance'] for r in simple_answer_similarity_scores])

0.2219844191505647

In [147]:
np.mean([r['summary'] for r in simple_answer_similarity_scores])

0.23630617871373105

In [148]:
np.mean([r['objectives'] for r in simple_answer_similarity_scores])

0.3591785686390749

In [149]:
np.mean([r['conclusions'] for r in simple_answer_similarity_scores])

0.30363927741350405

# Extra

In [162]:
idx = 810

In [163]:
generation_results[idx]["title"]

'Prevalência de transtornos mentais no estudantes de medicina da Universidade Federal do Rio de Janeiro'

In [164]:
generation_results[idx]["abstract"]

'Introdução: A prevalência de transtornos mentais encontrada entre os estudantes de medicina brasileiros tem sido mais alta do que a encontrada em estudantes de países desenvolvidos. Apesar de ser consenso na literatura que as prevalências encontradas em países em desenvolvimento são mais elevadas, torna-se relevante conhecer como esta dinâmica e seus fatores de risco se apresentam na realidade dos estudantes médicos da UFRJ. O objetivo deste estudo foi investigar a prevalência e fatores associados ao episódio depressivo maior e transtorno de ansiedade generalizada em estudantes de medicina da Universidade Federal do Rio de Janeiro, bem como perceber como estes estudantes avaliam o curso. Métodos: Foi realizado um estudo transversal em uma amostra representativa e aleatória de 296 estudantes (taxa de participação=91,4%), utilizando os seguintes instrumentos: PHQ-9 (Patient Health Questionare-9) para avaliação do episódio depressivo maior, GAD-7 (General Anxiety Disorder-7) para avaliaç

In [165]:
generation_results[idx]["query"]

'O objetivo da pesquisa foi investigar a prevalência e fatores associados ao episódio depressivo maior e transtorno de ansiedade generalizada em estudantes de medicina da Universidade Federal do Rio de Janeiro, bem como avaliar a percepção desses estudantes sobre o curso de medicina.'

In [178]:
generation_results[idx]["generation_results"]["results"][0]["doc"]["relevance_score"]

0.9999997

In [170]:
generation_results[idx]["generation_results"]["results"][0]["relevance"]

'O artigo é altamente relevante para a revisão de literatura do usuário, pois fornece informações detalhadas sobre a prevalência de transtornos mentais em estudantes de medicina, incluindo depressão e ansiedade, além de fatores associados e a percepção dos alunos sobre o curso. Esses dados podem contribuir significativamente para a pesquisa do usuário, fornecendo insights importantes sobre a saúde mental dos estudantes de medicina e possíveis intervenções para melhorar sua qualidade de vida.'

In [171]:
generation_results[idx]["generation_results"]["results"][0]["summary"]

'O artigo investigou a prevalência de transtornos mentais, como episódio depressivo maior e transtorno de ansiedade generalizada, em estudantes de medicina da Universidade Federal do Rio de Janeiro. Foram identificados altos índices de depressão e ansiedade, associados a fatores como gênero feminino, história de tratamento psiquiátrico e percepção negativa de estressores. Além disso, a percepção dos estudantes sobre o curso de medicina foi avaliada como tendo muitos problemas, indicando a necessidade de programas institucionais para promoção da saúde mental e melhoria da qualidade de vida dos alunos.'

In [172]:
generation_results[idx]["generation_results"]["results"][0]["objectives"]

'Investigar a prevalência de episódio depressivo maior e transtorno de ansiedade generalizada em estudantes de medicina da Universidade Federal do Rio de Janeiro, bem como avaliar a percepção desses estudantes sobre o curso.'

In [173]:
generation_results[idx]["generation_results"]["results"][0]["conclusions"]

'Os altos índices de depressão e ansiedade entre os estudantes de medicina da UFRJ apontam para a necessidade de implementação de programas institucionais que promovam a saúde mental e contribuam para a melhoria da qualidade de vida dos alunos. Além disso, a percepção dos estudantes sobre o curso indica a existência de diversos problemas que precisam ser abordados e melhorados.'